In [1]:
!pip install -U easynmt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)
  Created wheel for easynmt: filename=EasyNMT-2.0.2-py3-none-any.whl size=19898 sha256=2d91453b4c91ab90d562707e915c3a7819fdb7e766de51674c871406360cdc55
  Stored in directory: /root/.cache/pip/wheels/1c/5c/5d/d698bb79f4c9ddc0b910bb71d1ddb9048fb3bc7b0ed7ce40ea
  Created wheel for fasttext: filename=fasttext-0.9.3-cp312-cp312-linux_x86_64.whl size=4498212 sha256=04762340df3ee02b075e04685c039c006d66fc628c66858b4f8def73c1263205
  Stored in directory: /root/.cache/pip/wheels/20/27/95/a7baf1b435f1cbde017cabdf1e9688526d2b0e929255a359c6
Successfully built easynmt fasttext


**Experiment 1:** Using Opus-MT for Machine Translation

Creating an EasyNMT instance and loading a model. I am passing the model name I want to use and all needed files are downloaded and cached locally.

In [3]:
pip install langid


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=047c4d5e0b4a4ee5d68d693a6fd2df26ddf69dfedc6ac660ec11bd498dc984e1
  Stored in directory: /root/.cache/pip/wheels/3c/bc/9d/266e27289b9019680d65d9b608c37bff1eff565b001c977ec5
Successfully built langid


In [4]:
from easynmt import EasyNMT
model = EasyNMT('opus-mt', language_detection_method='langid')


In [6]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
sentences = ['In dieser Liste definieren wir mehrere Sätze.',
             'Jeder dieser Sätze wird dann in die Zielsprache übersetzt.',
             'Puede especificar en esta lista la oración en varios idiomas.',
             'El sistema detectará automáticamente el idioma y utilizará el modelo correcto.']
translations = model.translate(sentences, target_lang='en')

print("\n\nTranslations:")
for sent, trans in zip(sentences, translations):
  print(sent)
  print("=>", trans)
  print("")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]



Translations:
In dieser Liste definieren wir mehrere Sätze.
=> In this list we define several sentences.

Jeder dieser Sätze wird dann in die Zielsprache übersetzt.
=> Each of these sentences is then translated into the target language.

Puede especificar en esta lista la oración en varios idiomas.
=> You can specify the sentence in several languages in this list.

El sistema detectará automáticamente el idioma y utilizará el modelo correcto.
=> The system will automatically detect the language and use the correct model.



**Document Translation**

I can also pass longer documents (or list of documents) to the translate() method.

As Transformer models can only translate inputs up to 512 (or 1024) word pieces, we first perform sentence splitting. Then, each sentence is translated individually.

In [8]:
import tqdm
document = """Tokyo is the capital and most populous city of Japan, situated at the head of Tokyo Bay on the eastern coast of Honshu, the country’s largest island.
As of 2020, the Greater Tokyo Area had an estimated population of more than 37 million people, making it the most populous metropolitan region in the world.
Originally a small fishing village named Edo, the city became Japan’s political center in 1603 when Tokugawa Ieyasu established his shogunate there.
Edo was renamed Tokyo, meaning “Eastern Capital,” in 1868, when Emperor Meiji moved the imperial seat from Kyoto, marking the beginning of the Meiji Restoration and Japan’s rapid modernization.
Today, Tokyo is a major global financial and cultural hub, serving as the headquarters of many multinational corporations and international organizations.
The city blends cutting-edge technology with centuries-old tradition, where ancient temples stand beside futuristic skyscrapers and neon-lit shopping districts."""


print("Output:")
print(model.translate(document, target_lang='de'))

Output:


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

Tokio ist die Hauptstadt und bevölkerungsreichste Stadt Japans und liegt an der Spitze der Tokyo Bay an der Ostküste von Honshu, der größten Insel des Landes.
Seit 2020 hat das Großraumgebiet Tokio eine geschätzte Bevölkerung von mehr als 37 Millionen Menschen und ist damit die bevölkerungsreichste Metropolregion der Welt.
Ursprünglich ein kleines Fischerdorf namens Edo, wurde die Stadt 1603 Japans politisches Zentrum, als Tokugawa Ieyasu dort sein Shogunat gründete.
Edo wurde in Tokio umbenannt, was "Osthauptstadt" bedeutet, 1868, als Kaiser Meiji den kaiserlichen Sitz aus Kyoto verlegte und damit den Beginn der Meiji-Restaurierung und der raschen Modernisierung Japans markierte.
Heute ist Tokio ein wichtiges globales Finanz- und Kulturzentrum, das als Hauptsitz vieler multinationaler Konzerne und internationaler Organisationen dient.
Die Stadt verbindet modernste Technologie mit einer jahrhundertealten Tradition, in der antike Tempel neben futuristischen Wolkenkratzern und neonbeleuc

**Experiment 2**: Using mBART_50 for Machine Translation


In [9]:
from easynmt import EasyNMT
model = EasyNMT('mbart50_m2m')

24.9kB [00:00, 17.0MB/s]                   


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [10]:
sentences = ['In dieser Liste definieren wir mehrere Sätze.',
             'Jeder dieser Sätze wird dann in die Zielsprache übersetzt.',
             'Puede especificar en esta lista la oración en varios idiomas.',
             'El sistema detectará automáticamente el idioma y utilizará el modelo correcto.']
translations = model.translate(sentences, target_lang='en')

print("\n\nTranslations:")
for sent, trans in zip(sentences, translations):
  print(sent)
  print("=>", trans)
  print("")



Translations:
In dieser Liste definieren wir mehrere Sätze.
=> In this list we define several sentences.

Jeder dieser Sätze wird dann in die Zielsprache übersetzt.
=> Each of these sentences is then translated into the target language.

Puede especificar en esta lista la oración en varios idiomas.
=> You can specify in this list the speech in several languages.

El sistema detectará automáticamente el idioma y utilizará el modelo correcto.
=> The system will automatically detect the language and use the correct model.



**Experiment 3**: Using M2M_100 for Machine Translation


In [11]:
from easynmt import EasyNMT
model = EasyNMT('m2m_100_418M')

89.9kB [00:00, 490kB/s]


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [12]:
sentences = ['"इस सूची में, आप विभिन्न भाषाओं में वाक्य निर्दिष्ट कर सकते हैं।"']
translations = model.translate(sentences, target_lang='en')

print("\n\nTranslations:")
for sent, trans in zip(sentences, translations):
  print(sent)
  print("=>", trans)
  print("")



Translations:
"इस सूची में, आप विभिन्न भाषाओं में वाक्य निर्दिष्ट कर सकते हैं।"
=> "In this list, you can specify sentences in different languages."

